## Round 1 
* https://github.com/cal-itp/data-analyses/issues/1059
* cd rt_segment_speeds && pip install altair_transform && pip install -r requirements.txt && cd ../_shared_utils && make setup_env
* https://docs.google.com/document/d/1I1WiqlmU06W6iLCi7cZQrOCLILkrEfABEkcU0Jys7f0/edit
* https://route-speeds--cal-itp-data-analyses.netlify.app/name_bay-area-511-muni-schedule/0__report__name_bay-area-511-muni-schedule
* https://posit-dev.github.io/great-tables/get-started/nanoplots.html
* https://docs.pola.rs/py-polars/html/reference/api/polars.from_pandas.html
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/_rt_scheduled_utils.py
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/_threshold_utils.py

In [ ]:
%%capture
# import warnings
# warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import rt_dates, rt_utils

alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)
from typing import List, Union

from altair_transform.extract import extract_transform
from altair_transform.transform import visit
from altair_transform.utils import to_dataframe

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
name = "SBMTD Schedule"

In [ ]:
# %%capture_parameters
# name

### General Functions

In [ ]:
def labeling(word: str) -> str:
    return word.replace("_", " ").title().replace("N", "Total")

### Data

In [ ]:
# calitp-analytics-data/data-analyses/rt_vs_schedule/digest
df = pd.read_parquet(
    f"{RT_SCHED_GCS}digest/schedule_vp_metrics.parquet",
    filters=[[("name", "==", name)]],
)

In [ ]:
f"{RT_SCHED_GCS}digest/schedule_vp_metrics.parquet"

In [ ]:
most_recent_date = df.service_date.max()

In [ ]:
most_recent_date

In [ ]:
df.service_date.min()

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df["month_year"] = df.service_date.dt.strftime("%m/%Y")

### Test out Altair `extract_data`
* https://altair-viz.github.io/user_guide/transform/index.html

In [ ]:
__all__ = ["apply", "extract_data", "transform_chart"]

In [ ]:
def reverse_snakecase(df):
    """
    Clean up columns to remove underscores and spaces.
    """
    df.columns = df.columns.str.replace("_", " ").str.strip().str.title()
    return df

In [ ]:
def apply(
    df: pd.DataFrame,
    transform: Union[alt.Transform, List[alt.Transform]],
    inplace: bool = False,
) -> pd.DataFrame:
    """Apply transform or transforms to dataframe.

    Parameters
    ----------
    df : pd.DataFrame
    transform : list|dict
        A transform specification or list of transform specifications.
        Each specification must be valid according to Altair's transform
        schema.
    inplace : bool
        If True, then dataframe may be modified in-place. Default: False.

    Returns
    -------
    df_transformed : pd.DataFrame
        The transformed dataframe.

    Example
    -------
    >>> import pandas as pd
    >>> data = pd.DataFrame({'x': range(5), 'y': list('ABCAB')})
    >>> chart = alt.Chart(data).transform_aggregate(sum_x='sum(x)', groupby=['y'])
    >>> apply(data, chart.transform)
       y  sum_x
    0  A      3
    1  B      5
    2  C      2
    """
    if not inplace:
        df = df.copy()
    if transform is alt.Undefined:
        return df
    return visit(transform, df)

In [ ]:
def extract_data(
    chart: alt.Chart, apply_encoding_transforms: bool = True
) -> pd.DataFrame:
    """Extract transformed data from a chart.

    This only works with data and transform defined at the
    top level of the chart.

    Parameters
    ----------
    chart : alt.Chart
        The chart instance from which the data and transform
        will be extracted
    apply_encoding_transforms : bool
        If True (default), then apply transforms specified within an
        encoding as well as those specified directly in the transforms
        attribute.

    Returns
    -------
    df_transformed : pd.DataFrame
        The extracted and transformed dataframe.

    Example
    -------
    >>> import pandas as pd
    >>> data = pd.DataFrame({'x': range(5), 'y': list('ABCAB')})
    >>> chart = alt.Chart(data).mark_bar().encode(x='sum(x)', y='y')
    >>> extract_data(chart)
       y  sum_x
    0  A      3
    1  B      5
    2  C      2
    """
    if apply_encoding_transforms:
        chart = extract_transform(chart)
    return apply(to_dataframe(chart.data, chart), chart.transform)

In [ ]:
data = pd.DataFrame({"x": range(5), "y": list("ABCAB")})

In [ ]:
chart = alt.Chart(data).mark_bar().encode(x="sum(x)", y="y")

In [ ]:
chart

In [ ]:
extract_data(chart)

In [ ]:
def extract_data_altair(chart):
    chart_dict = chart.to_dict()
    encoding = chart_dict["datasets"]
    df1 = pd.DataFrame(encoding)

    column = df1.columns[0]
    normalized_df = pd.json_normalize(df1[column])
    # Combine the original DataFrame with the extracted values DataFrame
    df2 = pd.concat([df1.drop(column, axis=1), normalized_df], axis=1)
    return df2

### Monthly aggregated service hours by day_type, time_of_day

In [ ]:
from segment_speed_utils.project_vars import SCHED_GCS

In [ ]:
year = "2023"

In [ ]:
monthly_service_df = pd.read_parquet(
    f"{SCHED_GCS}scheduled_service_by_route_{year}.parquet",
    filters=[[("name", "==", name)]],
)

In [ ]:
monthly_service_df.shape

In [ ]:
monthly_service_df.sample()

In [ ]:
monthly_service_df.day_type.value_counts()

In [ ]:
monthly_service_df["full_date"] = pd.to_datetime(
    (monthly_service_df.month.astype(str) + "-" + monthly_service_df.year.astype(str))
)

In [ ]:
def tag_day(df: pd.DataFrame) -> pd.DataFrame:
    # Function to determine if a date is a weekend day or a weekday
    def which_day(date):
        if date == 1:
            return "Monday"
        elif date == 2:
            return "Tuesday"
        elif date == 3:
            return "Wednesday"
        elif date == 4:
            return "Thursday"
        elif date == 5:
            return "Friday"
        elif date == 6:
            return "Saturday"
        else:
            return "Sunday"

    # Apply the function to each value in the "service_date" column
    df["day_type"] = df["day_type"].apply(which_day)

    return df

In [ ]:
monthly_service_df.info()

In [ ]:
monthly_service_df = tag_day(monthly_service_df)

In [ ]:
monthly_service = (
    monthly_service_df.groupby(
        [
            "full_date",
            "month",
            "name",
            "day_type",
            "time_of_day",
        ]
    )
    .agg(
        {
            "ttl_service_hours": "mean",
        }
    )
    .reset_index()
)

In [ ]:
monthly_service.shape

In [ ]:
monthly_service.sample()

####  Fix
* Sort dropdown menu should have the fully spelled months?
* Dropdown menu does not like datetime values, pad single digit months with a 0 and concat it with the year?

In [ ]:
dropdown_list = sorted(monthly_service["full_date"].unique().tolist())

In [ ]:
initialize_first_val = sorted(dropdown_list)[0]

In [ ]:
initialize_first_val

In [ ]:
dropdown = alt.binding_select(options=dropdown_list, name=labeling("full_date"))

In [ ]:
def bar_chart_dropdown(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    offset_col: str,
    title: str,
    dropdown_col: str,
):
    dropdown_list = sorted(df[dropdown_col].unique().tolist())

    initialize_first_val = sorted(dropdown_list)[0]

    dropdown = alt.binding_select(options=dropdown_list, name=labeling(dropdown_col))

    selector = alt.selection_single(
        name=labeling(dropdown_col), fields=[dropdown_col], bind=dropdown
    )

    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({y_col}):Q")
    )

    chart = (
        alt.Chart(monthly_service)
        .mark_bar()
        .encode(
            x=alt.X(
                f"{x_col}:N",
                title="Day",
                scale=alt.Scale(
                    domain=[
                        "Monday",
                        "Tuesday",
                        "Wednesday",
                        "Thursday",
                        "Friday",
                        "Saturday",
                        "Sunday",
                    ]
                ),
            ),
            y=alt.Y(f"{y_col}:Q", title="Total Service Hours"),
            xOffset=f"{offset_col}:N",
            color=alt.Color(
                f"{offset_col}:N",
                scale=alt.Scale(
                    range=cp.CALITP_SEQUENTIAL_COLORS,
                ),
            ),
        )
    )
    chart = (chart + ruler).properties(title=title, width=600, height=400)
    chart = chart.add_params(selector).transform_filter(selector)
    # data = chart.transformed_data()

    display(chart)

In [ ]:
bar_chart_dropdown(
    monthly_service,
    "day_type",
    "ttl_service_hours",
    "time_of_day",
    "Average Service Hours by Month and Day for 2023",
    "month",
)

### Monthly Trends
* https://posit-dev.github.io/great-tables/articles/intro.html

In [ ]:
by_date_category = (
    pd.crosstab(
        df.service_date,
        df.sched_rt_category,
        values=df.n_scheduled_trips,
        aggfunc="sum",
    )
    .reset_index()
    .fillna(0)
)

In [ ]:
by_date_category.columns

In [ ]:
(
    gt.GT(by_date_category, rowname_col="service_date")
    .tab_header(
        title="Daily Trips by GTFS Availability",
        subtitle="Schedule only indicates the trip(s) were found only in static, schedule data. Vehicle Positions (VP) only indicates the trip(s) were found only in real-time data.",
    )
    .cols_label(
        schedule_only="Schedule Only",
        vp_only="VP Only",
        schedule_and_vp="Schedule and VP",
    )
    .fmt_integer(["schedule_only", "vp_only", "schedule_and_vp"])
    .tab_options(container_width="100%")
    .tab_options(table_font_size="12px")
)

In [ ]:
route_categories = (
    df[df.time_period == "all_day"]
    .groupby("sched_rt_category")
    .agg({"route_combined_name": "nunique"})
    .reset_index()
)

In [ ]:
route_categories.sched_rt_category = route_categories.sched_rt_category.str.replace(
    "_", " "
).str.title()

#### GTFS Availability
* Change Category values to something more understandable?

In [ ]:
(
    gt.GT(data=route_categories.dropna())
    .fmt_integer(columns=["route_combined_name"], compact=True)
    .cols_label(route_combined_name="Total Routes", sched_rt_category="Category")
    .tab_options(container_width="100%")
    .tab_header(
        title="Routes with GTFS Availability",
        subtitle="Schedule only indicates the route(s) were found only in static, schedule data. Vehicle Positions (VP) only indicates the route(s) were found only in real-time data.",
    )
    .tab_options(table_font_size="12px")
)

#### Route Stats

In [ ]:
route_merge_cols = ["route_combined_name", "direction_id"]

all_day_stats = df[
    (df.service_date == most_recent_date) & (df.time_period == "all_day")
][
    route_merge_cols
    + [
        "avg_scheduled_service_minutes",
        "avg_stop_miles",
        "n_scheduled_trips",
        "sched_rt_category",
    ]
]

In [ ]:
peak_stats = df[(df.service_date == most_recent_date) & (df.time_period == "peak")][
    route_merge_cols + ["speed_mph", "n_scheduled_trips", "frequency"]
].rename(
    columns={
        "speed_mph": "peak_avg_speed",
        "n_scheduled_trips": "peak_scheduled_trips",
        "frequency": "peak_hourly_freq",
    }
)

In [ ]:
offpeak_stats = df[
    (df.service_date == most_recent_date) & (df.time_period == "offpeak")
][route_merge_cols + ["speed_mph", "n_scheduled_trips", "frequency"]].rename(
    columns={
        "speed_mph": "offpeak_avg_speed",
        "n_scheduled_trips": "offpeak_scheduled_trips",
        "frequency": "offpeak_hourly_freq",
    }
)

In [ ]:
table_df = (
    pd.merge(
        all_day_stats,
        peak_stats,
        on=route_merge_cols,
    )
    .merge(offpeak_stats, on=route_merge_cols)
    .sort_values(["route_combined_name", "direction_id"])
    .reset_index(drop=True)
)

In [ ]:
table_df.columns

In [ ]:
table_df.sample()

In [ ]:
numeric_cols = table_df.select_dtypes(include="number").columns
table_df[numeric_cols] = table_df[numeric_cols].fillna(0)

In [ ]:
table_df.head()

#### Updating Already Made Charts
#### Scheduled Minutes...Can just be a table?

In [ ]:
sched_df = df[df.sched_rt_category != "vp_only"]
vp_df = df[df.sched_rt_category != "schedule_only"]

sched_service_chart = sched_df[sched_df.time_period == "all_day"]

In [ ]:
sched_service_chart.sample()

#### Grouped Bar Chart 
* Problems: No title shows up for the XOffset, have to email Altair people? It's confusing to not show the directions clearly
* Testing with `one_route`

In [ ]:
def grouped_bar_chart(
    df: pd.DataFrame, color_col: str, y_col: str, title: str, subtitle: str
):
    df = df.assign(
        time_period=df.time_period.str.replace("_", " ").str.title()
    ).reset_index(drop=True)
    
    df[y_col] = df[y_col].fillna(0).astype(int)
    tooltip_cols = [
        "direction_id",
        "time_period",
        "route_combined_name",
        "organization_name",
        "caltrans_district",
        "month_year",
        y_col,
    ]

    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({y_col}):Q")
    )

    chart = (
        alt.Chart(df)
        .mark_bar(size=10)
        .encode(
            x=alt.X(
                "yearmonthdate(service_date):O",
                title="Date",
                axis=alt.Axis(format="%b %Y"),
            ),
            y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
            xOffset=alt.X(f"direction_id:N", title="Direction ID"),
            color=alt.Color(
                f"{color_col}:N",
                title=labeling(color_col),
                scale=alt.Scale(
                    range=cp.CALITP_SEQUENTIAL_COLORS,
                ),
            ),
            tooltip=tooltip_cols,
        )
    )
    chart = (chart + ruler).properties(
        title={
            "text": [title],
            "subtitle": ["Broken out by Direction ID", subtitle],
        },
        width=500,
        height=300,
    )

    return chart

In [ ]:
one_route = sched_df.loc[sched_df.route_combined_name == "14 Montecito"]

In [ ]:
"""grouped_bar_chart(
            one_route,
            "service_date",
            "n_scheduled_trips",
            "direction_id",
            "Total Daily Trips",
        )"""

#### Avg Scheduled Minutes

In [ ]:
one_route[one_route.time_period == "all_day"].sample()

In [ ]:
grouped_bar_chart(
    df=one_route[one_route.time_period == "all_day"],
    color_col="direction_id",
    y_col="avg_scheduled_service_minutes",
    title="Average Scheduled Minutes",
    subtitle="The average minutes of scheduled service across all routes and time periods.",
)

#### Total Scheduled Trips

In [ ]:
scheduled_trips = grouped_bar_chart(
    df=one_route,
    color_col="time_period",
    y_col="n_scheduled_trips",
    title="Total Scheduled Trips",
    subtitle="The total number of scheduled trips for a route, differentiated by time periods.",
)

In [ ]:
scheduled_trips

#### Frequency
* Maybe shouldn't be a chart since there doesn't seem to be a lot of data for this across a lot of the routes?

In [ ]:
one_route.sample()

In [ ]:
one_route.frequency.value_counts()

In [ ]:
grouped_bar_chart(
    df=one_route,
    color_col="time_period",
    y_col="frequency",
    title="Total Scheduled Trips",
    subtitle="The number of times a route is run per hour.",
)

#### Speed MPH
* Needs a different type of chart.

In [ ]:
one_route_vp_df = vp_df.loc[vp_df.route_combined_name == "14 Montecito"]

In [ ]:
one_route_vp_df.shape

In [ ]:
# This doesn't work
def speed_chart(df: pd.DataFrame):
    dir_0 = df.loc[df.direction_id == 0]
    dir_1 = df.loc[df.direction_id == 1]

    def create_chart(df, direction: str):
        ruler = (
            alt.Chart(df)
            .mark_rule(color="red", strokeDash=[10, 7])
            .encode(y=f"mean(speed_mph):Q")
        )

        df.speed_mph = df.speed_mph.fillna(0).astype(int)
        tooltip_cols = [
            "direction_id",
            "time_period",
            "route_combined_name",
            "organization_name",
            "caltrans_district",
            "month_year",
            "speed_mph",
        ]
        chart = (
            alt.Chart(df)
            .mark_line(size=5)
            .encode(
                x=alt.X(
                    "service_date:N",
                    title="Date",
                ),
                y=alt.Y(
                    "speed_mph:Q", title="Speed MPH", scale=alt.Scale(domain=[5, 50])
                ),
                xOffset=alt.X(f"time_period:N", title="Time_Period"),
                color=alt.Color(
                    "time_period:N",
                    scale=alt.Scale(
                        range=cp.CALITP_SEQUENTIAL_COLORS,
                    ),
                ),
                tooltip=tooltip_cols,
            )
        )
        chart = (chart + ruler).properties(title=direction, width=300, height=400)
        return chart

    chart_0 = create_chart(dir_0, "Direction ID: 0")
    chart_1 = create_chart(dir_1, "Direction ID: 1")

    chart_list = [chart_0, chart_1]
    final_chart = alt.hconcat(*chart_list)
    final_chart = final_chart.properties(
        title={
            "text": ["Average Speed"],
            "subtitle": [
                "Broken out by Direction ID",
                "Average MPH a route reaches each month, broken out by time period and direction ID.",
            ],
        }
    )
    return final_chart

##### Editing Facet chart

In [ ]:
def base_route_chart(df: pd.DataFrame, y_col: str, title:str, subtitle:str) -> alt.Chart:
    """ """
    df = df.assign(
        time_period=df.time_period.str.replace("_", " ").str.title()
    ).reset_index(drop=True)

    selected_colors = [
        cp.CALITP_CATEGORY_BRIGHT_COLORS[0],  # blue
        cp.CALITP_CATEGORY_BRIGHT_COLORS[3],  # green
        cp.CALITP_CATEGORY_BOLD_COLORS[1],  # orange,
    ]

    # https://stackoverflow.com/questions/26454649/python-round-up-to-the-nearest-ten

    if "pct" in y_col:
        max_y = 1.2
    elif "per_minute" in y_col:
        max_y = round(df[y_col].max())
    else:
        max_y = round(df[y_col].max(), -1)

    df[f"y_col_str"] = df[y_col].astype(str)
    
    ruler = (
            alt.Chart(df)
            .mark_rule(color="red", strokeDash=[10, 7])
            .encode(y=f"mean(speed_mph):Q")
        )
        
    chart = (
        alt.Chart(df)
        .mark_line(size=5)
        .encode(
            x=alt.X(
                "yearmonthdate(service_date):O",
                title="Date",
                axis=alt.Axis(format="%b %Y"),
            ),
            y=alt.Y(
                f"{y_col}:Q", title=labeling(y_col), scale=alt.Scale(domain=[0, max_y])
            ),
            color=alt.Color(
                "time_period:N",
                title=labeling("time_period"),
                scale=alt.Scale(range=cp.CALITP_SEQUENTIAL_COLORS),
            ),
            tooltip=[
                "route_combined_name",
                "route_id",
                "direction_id",
                "time_period",
                "y_col_str",
            ],
        ))
        
    chart = (chart+ruler).properties(
        width = 300,
        height = 300)
    
    chart = chart.facet(
            column=alt.Column("direction_id:N", title=labeling("direction_id")),
        ).properties(title={
            "text": [title],
            "subtitle": [subtitle],
        }, )
    
    
    return chart

In [ ]:
base_route_chart(one_route_vp_df, "speed_mph", "my_title", "my_subtitle")

In [ ]:
# speed_chart(one_route_vp_df)

#### Putting it all together

In [ ]:
def filtered_route_charts3(
    df: pd.DataFrame,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """

    route_dropdown = alt.binding_select(
        options=sorted(df["route_combined_name"].unique().tolist()),
        name="Routes ",
    )

    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=["route_combined_name"],
        bind=route_dropdown,
    )

    sched_df = df[df.sched_rt_category != "vp_only"]
    vp_df = df[df.sched_rt_category != "schedule_only"]

    scheduled_trips = (
        grouped_bar_chart(
            df=sched_df,
            color_col="time_period",
            y_col="n_scheduled_trips",
            title="Total Scheduled Trips",
            subtitle="The total number of scheduled trips for a route, differentiated by time periods.",
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    speeds_chart = (
        base_route_chart(vp_df, "speed_mph", "my_title", "my_subtitle")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    chart_lists = [scheduled_trips, speeds_chart]
    chart = alt.vconcat(*chart_lists).resolve_scale(y="independent")

    display(chart)

In [ ]:
filtered_route_charts3(df)

#### Actual Faceted Charts (from Tiffany)
* Issues 
    * Faceting doesn't  with daily scheduled trip and average speed.

In [ ]:
def filtered_route_charts(
    df: pd.DataFrame,
    control_field: str,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    route_dropdown = alt.binding_select(
        options=sorted(df[control_field].unique().tolist()),
        name="Routes ",
    )

    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=[control_field],
        bind=route_dropdown,
    )

    sched_df = df[df.sched_rt_category != "vp_only"]
    vp_df = df[df.sched_rt_category != "schedule_only"]

    sched_service_chart = (
        base_route_chart(
            sched_df[sched_df.time_period == "all_day"], "avg_scheduled_service_minutes"
        )
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    sched_trips_chart = (
        base_route_chart(sched_df, "n_scheduled_trips")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    sched_freq_chart = (
        base_route_chart(sched_df, "frequency")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    speeds_chart = (
        base_route_chart(vp_df, "speed_mph")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    ping_density_chart = (
        base_route_chart(vp_df, "vp_per_minute")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    spatial_accuracy_chart = (
        base_route_chart(vp_df, "pct_in_shape")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    atleast1vp_chart = (
        base_route_chart(vp_df, "pct_rt_journey_atleast1_vp")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    atleast2vp_chart = (
        base_route_chart(vp_df, "pct_rt_journey_atleast2_vp")
        .add_params(route_selector)
        .transform_filter(route_selector)
    )

    chart_list = [
        sched_service_chart,
        sched_trips_chart,
        sched_freq_chart,
        speeds_chart,
        ping_density_chart,
        spatial_accuracy_chart,
        atleast1vp_chart,
        atleast2vp_chart,
    ]

    chart = alt.vconcat(*chart_list).resolve_scale(y="independent")

    return chart

In [ ]:
vp_df.columns

In [ ]:
filtered_route_charts(df, "route_combined_name")

#### Fake Faceted Chart  Doesn't work. The dropdowns don't impact the charts.

In [ ]:
def fake_faceted_chart2(df: pd.DataFrame, column_of_interest: str, title=str):

    ruler = (
        alt.Chart(df)
        .mark_rule(color="red", strokeDash=[10, 7])
        .encode(y=f"mean({column_of_interest}):Q")
    )

    dir_0 = df.loc[df.direction_id == 0]
    dir_1 = df.loc[df.direction_id == 1]
    chart_dir_0 = (
        alt.Chart(
            dir_0,
            title=alt.Title(
                title,
                subtitle="Direction 0",
            ),
        )
        .mark_bar()
        .encode(
            x=alt.X("month_year", title=labeling("month_year")),
            xOffset="time_period:N",
            y=alt.Y(f"{column_of_interest}:Q", title=labeling(column_of_interest)),
            color=alt.Color(
                "time_period:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
        )
        .properties(width=300, height=300)
    )
    chart_dir_1 = (
        alt.Chart(
            dir_1,
            title=alt.Title(
                "Title",
                color="white",
                subtitle="Direction 1",
            ),
        )
        .mark_bar()
        .encode(
            x=alt.X("month_year", title=labeling("month_year")),
            xOffset="time_period:N",
            y=alt.Y(f"{column_of_interest}:Q", axis=None),
            color=alt.Color(
                "time_period:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
        )
        .properties(width=300, height=300)
    )
    chart_dir_0 = chart_dir_0 + ruler
    chart_dir_1 = chart_dir_1 + ruler
    final_chart = alt.hconcat(chart_dir_0, chart_dir_1)
    return final_chart

In [ ]:
def fake_faceted_chart(
    df: pd.DataFrame,
    control_field: str,
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    dropdown_list = df["route_combined_name"].unique().tolist()
    input_dropdown = alt.binding_select(options=dropdown_list, name="Routes")
    selection = alt.selection_point(fields=["route_combined_name"], bind=input_dropdown)

    selection = alt.selection_point(fields=["route_combined_name"], bind=input_dropdown)
    sched_df = df[df.sched_rt_category != "vp_only"]
    vp_df = df[df.sched_rt_category != "schedule_only"]

    daily_trips = fake_faceted_chart2(
        sched_df, "n_scheduled_trips", "Total Scheduled Daily Trips"
    )
    frequency = fake_faceted_chart2(
        sched_df, "frequency", "Frequency of Trips per Hour"
    )
    avg_speeds = fake_faceted_chart2(vp_df, "speed_mph", "Average Speed")

    chart_lists = [daily_trips, frequency, avg_speeds]
    chart = (
        alt.vconcat(*chart_lists)
        .resolve_scale(y="independent")
        .add_params(selection)
        .transform_filter(selection)
    )
    display(chart)

In [ ]:
fake_faceted_chart(df, "route_combined_name")